<a href="https://colab.research.google.com/github/KhosrojerdiA/Transformer-MT/blob/main/Layer_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

the Qkv each of them split to 8 parts and each of them will be in ATTN block with vq and ai

multiply them and concatenate them to get seq*512 as out (end of multi head attention)

then. go to normalization

Layer(X + out]

Normalization: smaller range center zero. stable training and much more easier steps in backpropigation. faster and more stable tarining

layer nor: activation values in neurons in every layer is normalized such that values is center zero and maybe sd of 1.


updating parameters after each batch (for larger model with large parameteres to learn)



In [ ]:
inputs = torch.Tensor([[[0.2, 0.1, 0.3],
                        [0.5, 0.1, 0.1]]])

inputs.size()


torch.Size([1, 2, 3])

In [ ]:
B, S, E = inputs.size()
inputs = inputs.reshape(S, B, E)
inputs.size()

torch.Size([2, 1, 3])

In [ ]:
inputs

tensor([[[0.2000, 0.1000, 0.3000]],

        [[0.5000, 0.1000, 0.1000]]])

In [ ]:

parameter_shape = inputs.size()[-2:]
gamma = nn.Parameter(torch.ones(parameter_shape))
beta =  nn.Parameter(torch.zeros(parameter_shape))


gamma.size(), beta.size()

(torch.Size([1, 3]), torch.Size([1, 3]))

In [ ]:
dims = [-(i + 1) for i in range(len(parameter_shape))]
dims

[-1, -2]

In [ ]:

mean = inputs.mean(dim=dims, keepdim=True)
mean.size()


torch.Size([2, 1, 1])

In [ ]:

var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
epsilon = 1e-5
std = (var + epsilon).sqrt()
std

tensor([[[0.0817]],

        [[0.1886]]])

In [ ]:

y = (inputs - mean) / std
y


tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]])

In [ ]:

out = gamma * y + beta
out

tensor([[[ 0.0000, -1.2238,  1.2238]],

        [[ 1.4140, -0.7070, -0.7070]]], grad_fn=<AddBackward0>)

In [ ]:
#Class

import torch
from torch import nn

class LayerNormalization():
    def __init__(self, parameters_shape, eps=1e-5):
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, input):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        print(f"Mean \n ({mean.size()}): \n {mean}")
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        print(f"Standard Deviation \n ({std.size()}): \n {std}")
        y = (inputs - mean) / std
        print(f"y \n ({y.size()}) = \n {y}")
        out = self.gamma * y  + self.beta
        print(f"out \n ({out.size()}) = \n {out}")
        return out

In [ ]:
batch_size = 3
sentence_length = 5
embedding_dim = 8
inputs = torch.randn(sentence_length, batch_size, embedding_dim)

print(f"input \n ({inputs.size()}) = \n {inputs}")

input 
 (torch.Size([5, 3, 8])) = 
 tensor([[[ 2.4260,  0.9442, -0.9984,  1.1918,  0.5910, -1.3366,  0.4683,
          -0.0749],
         [ 0.3600, -0.0958, -1.6559,  0.1489,  1.0259,  1.5067, -1.3143,
           0.9772],
         [-0.7535, -0.6716,  1.6041, -0.7733, -0.4921,  0.5063,  0.7917,
           0.0783]],

        [[-0.4813, -1.4185,  0.0549,  0.2196, -0.2825,  1.2432, -0.7605,
          -2.3117],
         [-0.1968, -0.7716, -0.2591, -0.0277, -1.4751,  1.2612,  0.7635,
          -1.0877],
         [ 1.1633, -0.0233,  0.5616,  0.7687, -0.1166,  0.0241,  2.3081,
          -0.9795]],

        [[-0.2853,  1.7670, -1.1398, -0.2916, -1.2310, -0.3929, -0.9294,
          -2.0001],
         [ 2.0130, -0.2587,  1.2018,  0.5848, -1.1394, -1.4015,  0.7736,
          -1.1247],
         [ 0.6259, -0.3924, -0.9036,  0.2023,  2.9318,  2.2872,  0.8503,
           1.4413]],

        [[-0.7153, -1.1154,  0.1698,  1.5532,  0.4877, -0.3071, -0.7821,
          -0.7591],
         [ 0.4821, -0.6515, 

In [ ]:

inputs.size()

torch.Size([5, 3, 8])

In [ ]:
inputs.size()[-1:]

torch.Size([8])

In [ ]:
layer_norm = LayerNormalization(inputs.size()[-1:])
out = layer_norm.forward(inputs)

Mean 
 (torch.Size([5, 3, 1])): 
 tensor([[[ 0.4014],
         [ 0.1191],
         [ 0.0362]],

        [[-0.4671],
         [-0.2242],
         [ 0.4633]],

        [[-0.5629],
         [ 0.0811],
         [ 0.8803]],

        [[-0.1835],
         [ 0.4127],
         [-0.6310]],

        [[-0.0399],
         [ 0.1556],
         [-0.5188]]])
Standard Deviation 
 (torch.Size([5, 3, 1])): 
 tensor([[[1.1328],
         [1.0485],
         [0.8148]],

        [[1.0061],
         [0.8539],
         [0.9249]],

        [[1.0360],
         [1.1715],
         [1.2160]],

        [[0.8252],
         [0.8932],
         [0.6519]],

        [[0.8896],
         [0.7715],
         [0.8903]]])
y 
 (torch.Size([5, 3, 8])) = 
 tensor([[[ 1.7873,  0.4791, -1.2357,  0.6977,  0.1673, -1.5343,  0.0590,
          -0.4205],
         [ 0.2298, -0.2050, -1.6929,  0.0284,  0.8649,  1.3234, -1.3671,
           0.8184],
         [-0.9693, -0.8687,  1.9243, -0.9935, -0.6485,  0.5769,  0.9272,
           0.0516]],



In [ ]:
out[0].mean(), out[0].std()

(tensor(1.2418e-09, grad_fn=<MeanBackward0>),
 tensor(1.0215, grad_fn=<StdBackward0>))